### Get corresponding NTA demographics
Uses `all_rides_nta.csv` *(saved from `get_nta()`)* and `demographics.csv` *(from transit project folder)*

In [ ]:
all_rides = pd.read_csv('all_rides_nta.csv') # ride_type, interval, latitude, longitude, cluster, nta_code
demographics = pd.read_csv('demographics.csv')

In [ ]:
# Join all_rides table with demographics table on nta_code
all_rides = all_rides.merge(demographics, on='nta_code')

### Get NTA codes for each ride
Results saved in shared file `all_rides_nta.csv` and used to get corresponding NTA demographics

In [ ]:
import pandas as pd

In [ ]:
all_rides = pd.read_csv('all_rides.csv') # ride_type, interval, latitude, longitude, cluster
geographic = pd.read_csv('geographic.csv')

In [ ]:
# List of lists of tuples of polygon corners (x=longitude,y=latitude) for one NTA
nta_names = geographic.columns
nta_poly = []
for nta in nta_names:
    L = geographic[nta][~geographic[nta].isnull()]
    it = iter(L)
    nta_poly.append(list(zip(it,it)))

In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [ ]:
# Find NTA that contains coordinate
def get_nta(row):
    point = Point(row['longitude'],row['latitude'])
    for nta,poly in zip(nta_names,nta_poly):
        polygon = Polygon(poly)
        if polygon.contains(point):
            return nta

In [ ]:
#!pip install pandarallel # Parallelize Pandas' apply()
from pandarallel import pandarallel
pandarallel.initialize()

In [ ]:
%%time
all_rides['nta_code'] = all_rides.parallel_apply(get_nta, axis=1)
all_rides = all_rides.drop('Unnamed: 0', axis=1)
all_rides.to_csv('all_rides_nta.csv')